In [ ]:
import numpy as np
from IPython.display import display, clear_output

def cyk_algorithm(grammar, start_symbol, input_string):
    """
    Implementa el algoritmo CYK para una gramática en forma normal de Chomsky.

    Parameters:
        grammar (dict): Gramática libre de contexto en forma de diccionario,
                        donde las claves son símbolos no terminales y los valores
                        son listas de producciones.
        start_symbol (str): Símbolo inicial de la gramática.
        input_string (str): Cadena de entrada a analizar.
    """
    # Dividir la cadena de entrada en símbolos
    input_string = input_string.split()
    n = len(input_string)

    # Inicializar la matriz CYK (triangular superior)
    table = [[set() for _ in range(n)] for _ in range(n)]
    backpointers = [[{} for _ in range(n)] for _ in range(n)]  # Para construir el árbol

    # Paso 1: Llenar la diagonal con las producciones que generan cada símbolo terminal
    for j in range(n):
        applied_rules = []
        for lhs, rhs_list in grammar.items():
            for rhs in rhs_list:
                if len(rhs) == 1 and rhs[0] == input_string[j]:
                    table[j][j].add(lhs)
                    backpointers[j][j][lhs] = rhs[0]  # Guardar la regla aplicada
                    applied_rules.append(f"{lhs} -> {rhs}")
        #clear_output(wait=True)
        print("Estado actual de la matriz CYK:")
        display_matrix(table, n)
        print(f"Procesando celda [{j}, {j}] (símbolo: {input_string[j]})")
        if applied_rules:
            print("Reglas aplicadas:")
            for rule in applied_rules:
                print(f"  {rule}")
        else:
            print("No se aplicaron reglas en esta iteración.")
        input("Presiona Enter para continuar...")

    # Paso 2: Llenar las otras celdas del triángulo superior
    for length in range(2, n + 1):  # Longitud del segmento
        for start in range(n - length + 1):  # Índice inicial del segmento
            end = start + length - 1  # Índice final del segmento

            applied_rules = []
            for split in range(start, end):  # Punto de división
                left = table[start][split]
                right = table[split + 1][end]

                for lhs, rhs_list in grammar.items():
                    for rhs in rhs_list:
                        if len(rhs) == 2 and rhs[0] in left and rhs[1] in right:
                            table[start][end].add(lhs)
                            backpointers[start][end][lhs] = (split, rhs[0], rhs[1])  # Guardar divisiones
                            applied_rules.append(f"{lhs} -> {rhs} (usando [{start},{split}] y [{split+1},{end}])")

            #clear_output(wait=True)
            print("Estado actual de la matriz CYK:")
            display_matrix(table, n)
            print(f"Procesando celda [{start}, {end}], longitud {length}")
            if applied_rules:
                print("Reglas aplicadas:")
                for rule in applied_rules:
                    print(f"  {rule}")
            else:
                print("No se aplicaron reglas en esta iteración.")
            input("Presiona Enter para continuar...")

    # Resultado final
    #clear_output(wait=True)
    print("Estado final de la matriz CYK:")
    display_matrix(table, n)
    if start_symbol in table[0][n - 1]:
        print("\nLa cadena pertenece al lenguaje.")
        print("\nÁrbol sintáctico:")
        print_tree(backpointers, 0, n - 1, start_symbol, input_string)
    else:
        print("\nLa cadena NO pertenece al lenguaje.")

    return table

def display_matrix(table, n):
    """Muestra la matriz CYK en una tabla legible."""
    for row in range(n):
        print(" ".join(["{" + ",".join(sorted(cell)) + "}" if cell else "{}" for cell in table[row]]))
    print()

def print_tree(backpointers, start, end, symbol, input_string):
    """Construye y muestra el árbol sintáctico a partir de los punteros."""
    if start == end:  # Caso base: símbolo terminal
        print(f"{symbol} -> {input_string[start]}")
        return

    split, left_symbol, right_symbol = backpointers[start][end][symbol]
    print(f"{symbol} -> {left_symbol} {right_symbol}")
    print_tree(backpointers, start, split, left_symbol, input_string)
    print_tree(backpointers, split + 1, end, right_symbol, input_string)

# Ejemplo de uso
def main():
    # Gramática en forma normal de Chomsky
    grammar = {
        'O': [['SN', 'SV']],
        'SN': [['gato'], ['pescado'], ['perro'], ['pez'], ['Det', 'N']],
        'Det': [['el'], ['la'], ['una']],
        'N': [['gato'], ['pescado'], ['perro'], ['pez']],
        'SV': [['salta'], ['duerme'], ['Vt', 'SN']],
        'Vt': [['ve'], ['ama'], ['come']]
    }
    start_symbol = 'O'
    input_string = "el gato come pescado"

    cyk_algorithm(grammar, start_symbol, input_string)

if __name__ == "__main__":
    main()